In [1]:
!pip install mysql-connector-python

In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)  # es para enseñar todas las columnas del df
pd.set_option('display.max_rows', None)     # es para enseñar todas las filas del df


import warnings
warnings.simplefilter('ignore')   # es para quitar warnings

In [3]:
import mysql.connector as conn

# Importo todos archivos, exploro y modifico

### ACTOR

In [4]:
df_actor=pd.read_csv('../data/actor.csv')


In [5]:
# Elimino la columna 'last_update'

df_actor = df_actor.drop(['last_update'], axis=1)

In [6]:
df_actor.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   actor_id    200 non-null    int64 
 1   first_name  200 non-null    object
 2   last_name   200 non-null    object
dtypes: int64(1), object(2)
memory usage: 26.2 KB


In [7]:
len(df_actor.drop_duplicates())==len(df_actor)  # no hay duplicados

True

### CATEGORY

In [8]:
df_category=pd.read_csv('../data/category.csv')

In [9]:
# Elimino la columna 'last_update'

df_category = df_category.drop(['last_update'], axis=1)

In [10]:
# Crear una fila nueva con una nueva categoría.

df2 = pd.DataFrame({'category_id': [99],
                    'name' : ['Unknown']})

df_category = pd.concat([df_category, df2], ignore_index = True, axis = 0)

### FILM

In [11]:
df_film=pd.read_csv('../data/film.csv')

### INVENTORY

In [12]:
df_inventory=pd.read_csv('../data/inventory.csv')

In [13]:
# Elimino la columna 'last_update'

df_inventory = df_inventory.drop(['last_update'], axis=1)

### LANGUAGE

In [14]:
df_language=pd.read_csv('../data/language.csv')


In [15]:
# Elimino la columna 'last_update'

df_language = df_language.drop(['last_update'], axis=1)

### RENTAL

In [16]:
df_rental=pd.read_csv('../data/rental.csv')

In [17]:
# Cambio el 'return_date' a formato 'date'

df_rental.rental_date = pd.to_datetime(df_rental.rental_date, format= '%Y-%m-%d')
df_rental.return_date = pd.to_datetime(df_rental.return_date, format= '%Y-%m-%d')

In [18]:
# Elimino la columna 'last_update'

df_rental = df_rental.drop(['last_update'], axis=1)

### OLD_HDD - Relaciones titulo, actor y categoría

In [19]:
df_old_HDD=pd.read_csv('../data/old_HDD.csv')

In [20]:
df_old_HDD.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   first_name    1000 non-null   object
 1   last_name     1000 non-null   object
 2   title         1000 non-null   object
 3   release_year  1000 non-null   int64 
 4   category_id   1000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 207.6 KB


# Modificar y limpiar

## Modifico tabla film

In [21]:
# Hago un left join en 'df_film' para obtener 'category_id', haciendo que en films y old_HDD el index sea title.


film = df_film.set_index('title').join(df_old_HDD[["title", "category_id"]].set_index('title'), lsuffix='', rsuffix='_dcha', how='left')


In [22]:
# Le elimino duplicado que se generan

film.drop_duplicates(inplace=True)

In [23]:
# Le reseteo el indice

film.reset_index(inplace=True)

In [24]:
# Relleno en la tabla film la columna category_id con 99 

film.category_id.fillna(99, inplace=True)

# Modificar la columna de category_id

film.category_id = film.category_id.astype(dtype='int64')

In [25]:
# Elimino la columna de original_language_id

film.drop('original_language_id', axis=1, inplace=True)

In [26]:
# Elimino la columna 'last_update'

film = film.drop(['last_update'], axis=1)

## Creo tabla actor_film

In [27]:
actor = df_actor.copy(deep=True)

In [28]:
# Creo una columna y concateno first name y last name

actor['name'] = actor['first_name'] + ' ' + actor['last_name']

In [29]:
hdd = df_old_HDD

In [30]:
# Creo una columna nueva y concateno first name y last name

hdd['name'] = hdd['first_name'] + ' ' + hdd['last_name']

In [31]:
# Creo la tabla 'actor_film' que voy a utilizar como relación many to many entre actor y film.

# Primero uso hdd para hacer un join a través del indice 'title' y obtener 'film_id'

actor_film = hdd.set_index('title').join(film[["title", "film_id"]].set_index('title'), lsuffix='', rsuffix='_dcha', how='left')


In [32]:
# Elimino duplicados por si se generaran

actor_film.drop_duplicates(inplace=True)

In [33]:
# Reseteo el indice para que la columna 'title' vuelva a ser una columna

actor_film.reset_index(inplace=True)

In [34]:
# Primero uso 'actor_film' para hacer un join a través del indice 'name' y obtener 'actor_id'

actor_film = actor_film.set_index('name').join(actor[["name", "actor_id"]].set_index('name'), lsuffix='', rsuffix='_dcha', how='left')

In [35]:
# Elimino duplicados y reseteo el índice

actor_film.drop_duplicates(inplace=True)
actor_film.reset_index(inplace=True)

In [36]:
# Sobreescribo 'actor_film' solo con las columnas de 'actor_id' y 'film_id'

actor_film = actor_film[['actor_id', 'film_id']]

## Creo e importo tablas nuevas

In [37]:
# Creo 'df_store' dataframe

store = {'store_id': [1, 2], 'name': ['Blockbuster MIA','Blockbuster Orlando' ], 'city': ['Miami', 'Orlando']}  
  
df_store = pd.DataFrame(store)

In [38]:
# Creo 'df_staff' dataframe

staff = {'staff_id': [1, 2], 'firstname': ['Barack','Donald' ], 'lastname': ['Obama', 'Trump'], 'salary': [35000, 35000]}  
  
df_staff = pd.DataFrame(staff)  


In [39]:
# Importo un random csv de 'customers' y me creo tabla 'id_customer' 

df_customer=pd.read_csv('../data/customers.csv')

## Exporto todas los dataframe a carpeta new_data

In [65]:
df_category.to_csv('../data/new_data/category.csv', sep= ",", index=False)

df_language.to_csv('../data/new_data/language.csv', sep= ",", index=False)

df_actor.to_csv('../data/new_data/actor.csv', sep= ",", index=False)

actor_film.to_csv('../data/new_data/actor_film.csv', sep= ",", index=False)

film.to_csv('../data/new_data/film.csv', sep= ",", index=False)

df_inventory.to_csv('../data/new_data/inventory.csv', sep= ",", index=False)

df_rental.to_csv('../data/new_data/rental.csv', sep= ",", index=False)

df_store.to_csv('../data/new_data/store.csv', sep= ",", index=False)

df_customer.to_csv('../data/new_data/customer.csv', sep= ",", index=False)


df_staff.to_csv('../data/new_data/staff.csv', sep= ",", index=False)

# Crear base de datos en mySQL

## Creo conexion y base de datos

In [40]:
df_rental.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1


In [41]:
df_inventory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   inventory_id  1000 non-null   int64
 1   film_id       1000 non-null   int64
 2   store_id      1000 non-null   int64
dtypes: int64(3)
memory usage: 23.6 KB


In [42]:
# Leo la contraseña de github

password = "".join(open('../text/mysql_pass.txt', "r").readlines())


In [43]:
# Conecta a una base de datos mysql y activa el cursor/motor

password = "".join(open('../text/mysql_pass.txt', "r").readlines()) # Me traigo el password de mysql

crea_db=conn.connect(host='localhost', user='root', passwd=password)

cursor=crea_db.cursor()



In [44]:
# Crear base de datos

cursor.execute('drop database if exists blockbuster;')
cursor.execute('create database blockbuster;')

In [45]:
# conecta a una base de datos

db=conn.connect(host='localhost', user='root', passwd=password, database='blockbuster')

cursor=db.cursor()

In [46]:
# check si me creo la base de datos

cursor.execute('show databases;')

for x in cursor:
    print(x)

('blockbuster',)
('information_schema',)
('mydb',)
('mysql',)
('performance_schema',)
('publications',)
('sys',)


## Creo tablas con sus caracteristicas

In [47]:
# Crear tabla 'actor'


cursor.execute('drop table if exists actor;')



tabla='''

        create table actor (
            
            actor_id int primary key,
            first_name varchar(50),
            last_name  varchar(50)
        );

'''


cursor.execute(tabla)




In [48]:
# Crear tabla 'language'

cursor.execute('drop table if exists language')

tabla= '''
            create table language (
            
            language_id int primary key,
            name varchar(50)         
            
            );
'''
cursor.execute(tabla)


In [49]:
# Crear tabla 'category'

cursor.execute('drop table if exists category')

tabla = '''
            create table category(
            category_id int primary key,
            name varchar(50)
            
            );
'''

cursor.execute(tabla)

In [50]:
# Crear tabla 'film'

cursor.execute('drop table if exists film')

tabla = '''
            create table film(
            
            film_id int primary key,
            title varchar(50),
            description varchar(250),
            release_year int,
            rental_duration int,
            rental_rate float,
            length int,
            replacement_cost float,
            special_features varchar(250),
            language_id int,
            category_id int
            
            );
'''

cursor.execute(tabla)

In [51]:
# Crear tabla 'inventory'

cursor.execute('drop table if exists inventory')

tabla = '''
            create table inventory(
            
            inventory_id int primary key,
            film_id int,
            store_id int
            
            );
'''

cursor.execute(tabla)

In [52]:
# Crear tabla 'rental'

cursor.execute('drop table if exists rental')

tabla = '''
            create table rental(
            
            rental_id int primary key,
            rental_date datetime,
            return_date datetime,
            inventory_id int,
            customer_id int,
            staff_id int
            
            );
'''

cursor.execute(tabla)

In [53]:
# Creo tabla 'store'

cursor.execute('drop table if exists store')

tabla = ''' 
            create table store(
            
            store_id int primary key,
            name varchar(50),
            city varchar(50)
    
            );

'''

cursor.execute(tabla)

In [54]:
# Creo tabla 'customer'

cursor.execute('drop table if exists customer')

tabla = '''
            create table customer(
            
            customer_id int primary key,
            firstname varchar(50),
            lastname varchar(50),
            email varchar(100)
            
            );


'''
cursor.execute(tabla)

In [55]:
# Creo tabla 'staff'

cursor.execute('drop table if exists staff')

tabla = '''
            create table staff(
            
            staff_id int primary key,
            firstname varchar(50),
            lastname varchar(50),
            salary int
            
            );

'''

cursor.execute(tabla)

In [56]:
# Creo la tabla 'actor_film' que hará de tabla intermedia en la relación many to many entre 'actor' y 'film'

cursor.execute('drop table if exists actor_film')

tabla = ''' 
            create table actor_film(
            
            actor_id int not null,
            film_id int not null,
            primary key(actor_id, film_id)
            
            
            );


'''

cursor.execute(tabla)

### Revisara relacion many to many

## Creo relaciones entre tablas

In [58]:
# Creo relaciones entre tabla 'film' y 'category' y 'language'



relation = '''
        
            alter table film
            add foreign key(category_id)
            references category(category_id)
            on delete set null;
        
            alter table film
            add foreign key(language_id)
            references language(language_id)
            on delete set null;
        
        
'''

cursor.execute(relation)


In [61]:
# Creo relaciones entre tabla 'inventory' y 'film' y 'store'

# Me genera un error
# DatabaseError: 2014 (HY000): Commands out of sync; you can't run this command now
# No se que mierda del buffer con las queries

'''
            alter table inventory
            add foreign key(film_id)
            references film(film_id)
            on delete set null;



            alter table inventory
            add foreign key(store_id)
            references store(store_id)
            on delete set null;
   
'''
# Revisar con Yona


'\n            alter table inventory\n            add foreign key(film_id)\n            references film(film_id)\n            on delete set null;\n\n\n\n            alter table inventory\n            add foreign key(store_id)\n            references store(store_id)\n            on delete set null;\n   \n'

In [ ]:
# Creo relaciones entre tabla 'rental' y 'inventory', 'customer' y 'staff'


'''        
            alter table rental
            add foreign key(inventory_id)
            references inventory(inventory_id)
            on delete set null;



            alter table rental
            add foreign key(customer_id)
            references customer(customer_id)
            on delete set null;
            
            alter table rental
            add foreign key(staff_id)
            references staff(staff_id)
            on delete set null;
            
        '''

In [ ]:
# Creo relaciones entre tabla 'actor_film' y 'actor' y 'film'

# Relacion many to many

'''
                alter table actor_film
                add constraint 
                fk_actorfilm__actor foreign key(actor_id) references actor(actor_id);

                alter table actor_film
                add constraint 
                fk_actorfilm__film foreign key(film_id) references film(film_id);



'''


